In [1]:
from flask import Flask
from flask_restful import Api, Resource
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import pandas as pd
import requests

app = Flask(__name__)
dash_main = dash.Dash(__name__, server=app)
all_data = pd.DataFrame(columns=["id", "room_id/id", "noted_date", "temp", "out/in"])
data_list = []

# 獲取資料
def loadData_GLOB():
    global all_data
    response = requests.get('http://localhost:5000/stream_csv')
    # 检查响应是否成功
    if response.status_code == 200:
        # 解析流式数据
        lines = response.text.strip().split('\n')
        data_list = [line.split(",") for line in lines]
        # 创建 DataFrame
        df = pd.DataFrame(data_list, columns=["id", "room_id/id", "noted_date", "temp", "out/in"])
        # 数据清理
        df['noted_date'] = df['noted_date'].str.strip('"')
        df['temp'] = df['temp'].str.strip('"')
        df['out/in'] = df['out/in'].str.replace('"', '')
        df['out/in'] = df['out/in'].str.replace(']', '')
        # 将 noted_date 转换为 datetime
        df['noted_date'] = pd.to_datetime(df['noted_date'], format='%d-%m-%Y %H:%M')
        # 删除重复行
        df.drop_duplicates(subset=['out/in', 'noted_date'], keep='first', inplace=True)

        # 將新資料附加到全局 DataFrame 上
        all_data = pd.concat([all_data, df], ignore_index=True)
    else:
        print("Failed to retrieve data. Status code:", response.status_code)


def filtered_data(roomid,Yearid,Monthid):
    global all_data
    loadData_GLOB()
    data1 = all_data.loc[all_data['out/in'] == roomid].drop('out/in', axis=1)
    data2=data1.loc[data1['noted_date'].dt.year  == Yearid]
    data=data2.loc[data2['noted_date'].dt.month == Monthid]
    return data

loadData_GLOB()
roomloc = all_data['out/in'].unique()  
roomloc.sort()  
yearloc = all_data['noted_date'].dt.year.unique()
monthloc = all_data['noted_date'].dt.month.unique()

# 定義布局
dash_main.layout = html.Div(
    style={'font-family': "Courier New, monospace"},
    children=[
        html.H1('IOT-溫度趨勢'),
        html.Div(className="row", children=[
            html.P('資料筆數:'), html.P(id="df_count_text"),
            html.H5('ROOMloc'),
            dcc.Dropdown(
                id='roomid',
                options=[{'label': loc, 'value': loc} for loc in roomloc],
                value=roomloc
            ),
            html.H5('Year''s'),
            dcc.Dropdown(
                id='Yearid',
                options=[{'label': year, 'value': year} for year in yearloc],
                value=yearloc
            ),
            html.H5('Month'),
            dcc.Dropdown(
                id='Monthid',
                options=[{'label': month, 'value': month} for month in monthloc],
                value=monthloc
            )
        ]),
        dcc.Graph(id="plot_new_metrics", config={'displayModeBar': False}),
        dcc.Interval(id='interval-component', interval=1000, n_intervals=0)
    ]
)


@dash_main.callback(
    [Output('plot_new_metrics', 'figure'), Output('df_count_text', 'children')],
    [Input('interval-component', 'n_intervals'),Input('roomid', 'value'), Input('Yearid', 'value'), Input('Monthid', 'value')]
)
def update_plots(n,roomid,Yearid,Monthid):
    #global all_data
    #loadData_GLOB()
    
    data = filtered_data(roomid,Yearid,Monthid)
    
    df_count_text = len(data.index)
    # 把剛拿到的 data 重新定義 figure
    fig = go.Figure(data=[go.Scatter(x=data['noted_date'], y=data['temp'], mode='lines')])
    # 定義 Dash 標題，X軸、Y軸
    fig.update_layout(title='Temperature Over Time', xaxis_title='Date', yaxis_title='Temperature')
    return fig,df_count_text


@dash_main.callback(
    [Output('roomid', 'options'), Output('Yearid', 'options'), Output('Monthid', 'options')],
    [Input('roomid', 'value'), Input('Yearid', 'value'), Input('Monthid', 'value')]  # 監聽下拉式選單的值
)
def update_options(room_value, year_value, month_value):
    global all_data
    roomloc = all_data['out/in'].unique()  # 把從 allData() 裡回傳的 pd，'out/in' 欄位移除重複
    roomloc.sort()  # 排序
    yearloc = all_data['noted_date'].dt.year.unique()
    monthloc = all_data['noted_date'].dt.month.unique()

    return [{'label': loc, 'value': loc} for loc in roomloc], [{'label': year, 'value': year} for year in yearloc], [{'label': month, 'value': month} for month in monthloc]


if __name__ == "__main__":
    app.run(port=8101)


C:\Users\king8\AppData\Local\Temp\ipykernel_27964\3072171528.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\king8\AppData\Local\Temp\ipykernel_27964\3072171528.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\king8\AppData\Local\Temp\ipykernel_27964\3072171528.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

 * Running on http://127.0.0.1:8101
Press CTRL+C to quit
127.0.0.1 - - [09/May/2024 21:22:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 21:22:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 21:22:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 21:22:44] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2024 21:22:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 21:22:44] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2024 21:22:44] "GET /_dash-component-suites/plotl